In [41]:
import sys
import os
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/mL/results/ridge/acc/'
import results

#SS
SS=pd.read_csv(outDir+'SS/folds/allFolds.csv')
BS=pd.read_csv(outDir+'BS/folds/allFolds.csv')


#BS=pd.read_csv(outDir+'results/ridge/acc/BS/acc.csv', usecols=[1,2,3,4,5])

#SS=pd.read_csv(outDir+'results/ridge/acc/SS/acc.csv', usecols=[1,2,3,4])

#motor
SS_motor=SS[SS.train=='motor'].acc.values
BS_motor=BS[BS.train_task=='motor'].acc.values

#mixed
SS_mixed=SS[SS.train=='mixed'].acc.values
BS_mixed=BS[BS.train_task=='mixed'].acc.values

#memory
SS_mem=SS[SS.train=='mem'].acc.values
BS_mem=BS[BS.train_task=='mem'].acc.values


In [49]:
stats.ttest_ind(SS_motor, BS_motor, equal_var = False)

Ttest_indResult(statistic=0.4580957623050814, pvalue=0.6473239862027558)

In [50]:
stats.ttest_ind(SS_mixed, BS_mixed, equal_var = False)

Ttest_indResult(statistic=26.525862574829606, pvalue=4.157442036651116e-85)

In [51]:
stats.ttest_ind(SS_mem, BS_mem, equal_var = False)

Ttest_indResult(statistic=4.378975855773864, pvalue=1.8125319013851326e-05)

In [69]:
#motor train, mem test
SS_motorMem=SS[(SS['train']=='motor') & (SS['test']=='mem')].acc.values
BS_motorMem=BS[(BS['train_task']=='motor') & (BS['test_task']=='mem')].acc.values
#motor train mix test
SS_motorMix=SS[(SS['train']=='motor') & (SS['test']=='mixed')].acc.values
BS_motorMix=BS[(BS['train_task']=='motor') & (BS['test_task']=='mixed')].acc.values

#mixed train, mem test
SS_mixMem=SS[(SS['train']=='mixed') & (SS['test']=='mem')].acc.values
BS_mixMem=BS[(BS['train_task']=='mixed') & (BS['test_task']=='mem')].acc.values
#mixed train motor test
SS_mixMotor=SS[(SS['train']=='mixed') & (SS['test']=='motor')].acc.values
BS_mixMotor=BS[(BS['train_task']=='mixed') & (BS['test_task']=='motor')].acc.values

#memory train, mix test
SS_memMix=SS[(SS['train']=='mem') & (SS['test']=='mixed')].acc.values
BS_memMix=BS[(BS['train_task']=='mem') & (BS['test_task']=='mixed')].acc.values
#memory train motor test
SS_memMotor=SS[(SS['train']=='mem') & (SS['test']=='motor')].acc.values
BS_memMotor=BS[(BS['train_task']=='mem') & (BS['test_task']=='motor')].acc.values


In [75]:
stats.ttest_ind(SS_motorMem, BS_motorMem, equal_var = False)


Ttest_indResult(statistic=0.9487127757648736, pvalue=0.34477658986204773)

In [74]:
stats.ttest_ind(SS_motorMix, BS_motorMix, equal_var = False)

Ttest_indResult(statistic=-0.34990584198770214, pvalue=0.7270597682875242)

In [76]:
stats.ttest_ind(SS_mixMem, BS_mixMem, equal_var = False)

Ttest_indResult(statistic=19.82408262471779, pvalue=1.851419802044856e-48)

In [77]:
stats.ttest_ind(SS_mixMotor, BS_mixMotor, equal_var = False)

Ttest_indResult(statistic=17.791152999069528, pvalue=3.623471345430174e-39)

In [78]:
stats.ttest_ind(SS_memMix, BS_memMix, equal_var = False)

Ttest_indResult(statistic=2.4830770282481436, pvalue=0.014380103556397805)

In [79]:
stats.ttest_ind(SS_memMotor, BS_memMotor, equal_var = False)

Ttest_indResult(statistic=3.826524046275118, pvalue=0.00021511136616734808)

In [84]:
SS_motorMem.mean()
SS_motorMem.std()

BS_motorMem.mean()
#BS_motorMem.std()

0.6201785714285714

In [94]:
#SS_motorMix.mean()
#SS_motorMix.std()

#BS_motorMix.mean()
BS_motorMix.std()

0.11334644719547557

In [96]:
#SS_memMotor.mean()
#SS_memMotor.std()

#BS_memMotor.mean()
BS_memMotor.std()

0.13605642961953812

In [100]:
#SS_memMix.mean()
#SS_memMix.std()

#BS_memMix.mean()
BS_memMix.std()

0.13606718098919718

In [104]:
#SS_mixMem.mean()
#SS_mixMem.std()

#BS_mixMem.mean()
BS_mixMem.std()

0.1559274988101973

In [108]:
#SS_mixMotor.mean()
#SS_mixMotor.std()

#BS_mixMotor.mean()
BS_mixMotor.std()

0.1503079057064436